# Notebook to generate the dataset and push to HuggingFace hub
#### What we need to do for data prep on target:
1. Only get notes on subjects that have a discharge summary
2. Further filter to only get hadm_ids/subjects that have a single discharge summary report (some can be addendums, we will ignore those notes)
3. Drop addendums
4. Find headers
5. Get target text ("BRIEF HOSPITAL COURSE" or "HOSPITAL COURSE")
6. Drop subjects/hadm_ids that do not have this section or have them both (why would they have both? only 34 samples so just drop)


In [1]:
import pandas as pd
import numpy as np
from datasets import Dataset, load_dataset
import os
from src.helpers import get_discharge_headers, get_target_text
from src.query_athena import QueryAthena

/home/dmacres/anaconda3/envs/gendis/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Get NoteEvents Table

In [2]:
athena = QueryAthena()

In [3]:
query = "select * from noteevents"
noteevents = athena.run_query(query)

Execution ID: d01a8a84-fe2b-48c9-baf9-5831e469e828
QUEUED


KeyboardInterrupt: 

In [ ]:
noteevents.head()

## 1. Get subjects with discharge summaries

In [ ]:
ds_by_subj = noteevents.groupby('subject_id').agg({'category': lambda x: True if 'Discharge summary' in list(x) else False}).reset_index().rename(columns = {'category': 'ds_present'})
ds_by_subj.ds_present.value_counts()

In [ ]:
subjs_w_ds = ds_by_subj[ds_by_subj.ds_present == True].subject_id.tolist()
len(subjs_w_ds)

In [ ]:
ne_ds = noteevents[noteevents.subject_id.isin(subjs_w_ds)].reset_index(drop = True)

In [ ]:
ne_ds.shape

In [9]:
ne_ds.head()

,row_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,text
0,2004671,24291,148229.0,2163-01-19 00:00:00.000,2163-01-19 18:39:00.000,2163-01-19 18:41:00.000,Nursing/other,Report,15522.0,NaN,Nursing NICU note Addendum.\nMother in this ev...
1,587175,69322,192784.0,2176-07-21 00:00:00.000,2176-07-21 09:36:00.000,2176-07-21 09:36:11.000,Physician,Generic Note,18780.0,NaN,TITLE:\n CVICU\n HPI:\n HD5\n [**7-21*...
2,2004672,24291,148229.0,2163-01-20 00:00:00.000,2163-01-20 06:45:00.000,2163-01-20 07:00:00.000,Nursing/other,Report,16024.0,NaN,NPN 1900-0700\n\n\n1. FEN: TF=130cc/k/day B...
3,2002347,24131,117291.0,2174-10-11 00:00:00.000,2174-10-11 08:08:00.000,2174-10-11 08:12:00.000,Nursing/other,Report,20888.0,NaN,[**Location (un) 369**]/NEON DOL 5 CGA 34 [**5...
4,2002348,24131,117291.0,2174-10-11 00:00:00.000,2174-10-11 15:20:00.000,2174-10-11 15:27:00.000,Nursing/other,Report,18389.0,NaN,"NPNote\n\n\n#1.Remains in Ra ir, BBS clear, eq..."


## 2. Get HADM_IDs where there is only 1 discharge summary report
Subjects can have multiple HADM_IDs, but HADM_ID is unique

In [10]:
pivot_ne_ds = ne_ds[['subject_id', 'hadm_id', 'category', 'description']].pivot_table(index=['subject_id', 'hadm_id', 'description'], columns='category', aggfunc='size', fill_value=0).reset_index()
pivot_ne_ds.head()

category,subject_id,hadm_id,description,Case Management,Consult,Discharge summary,ECG,Echo,General,Nursing,Nursing/other,Nutrition,Pharmacy,Physician,Radiology,Rehab Services,Respiratory,Social Work
0,3,145834.0,CHEST (PORTABLE AP),0,0,0,0,0,0,0,0,0,0,0,12,0,0,0
1,3,145834.0,CT ABDOMEN W/O CONTRAST,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,3,145834.0,Report,0,0,1,9,3,0,0,19,0,0,0,0,0,0,0
3,3,145834.0,VIDEO OROPHARYNGEAL SWALLOW,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,4,185777.0,CHEST (PORTABLE AP),0,0,0,0,0,0,0,0,0,0,0,6,0,0,0


In [11]:
target_hadm_ids = pivot_ne_ds[(pivot_ne_ds.description.str.lower()=='report') & (pivot_ne_ds['Discharge summary']==1)].hadm_id.tolist()

In [12]:
noteevents2 = noteevents[noteevents.hadm_id.isin(target_hadm_ids)]
noteevents2[noteevents2.category.str.lower()=='discharge summary'].description.value_counts()

description
Report      50563
Addendum     3962
Name: count, dtype: int64

## 3. Drop Addendums

In [13]:
noteevents2 = noteevents2[~((noteevents2.category.str.lower() == 'discharge summary') & (noteevents2.description.str.lower() == 'addendum'))].reset_index(drop=True)
noteevents2[noteevents2.category.str.lower()=='discharge summary'].description.value_counts()

description
Report    50563
Name: count, dtype: int64

## 4. Find Headers

In [14]:
noteevents2.loc[noteevents2.category == 'Discharge summary', 'headers'] = noteevents2.loc[noteevents2.category == 'Discharge summary'].text.apply(lambda x: get_discharge_headers(x))

In [15]:
noteevents2.head()

,row_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,text,headers
0,587175,69322,192784.0,2176-07-21 00:00:00.000,2176-07-21 09:36:00.000,2176-07-21 09:36:11.000,Physician,Generic Note,18780.0,NaN,TITLE:\n CVICU\n HPI:\n HD5\n [**7-21*...,NaN
1,2002347,24131,117291.0,2174-10-11 00:00:00.000,2174-10-11 08:08:00.000,2174-10-11 08:12:00.000,Nursing/other,Report,20888.0,NaN,[**Location (un) 369**]/NEON DOL 5 CGA 34 [**5...,NaN
2,2002348,24131,117291.0,2174-10-11 00:00:00.000,2174-10-11 15:20:00.000,2174-10-11 15:27:00.000,Nursing/other,Report,18389.0,NaN,"NPNote\n\n\n#1.Remains in Ra ir, BBS clear, eq...",NaN
3,2002349,24131,117291.0,2174-10-11 00:00:00.000,2174-10-11 18:03:00.000,2174-10-11 18:04:00.000,Nursing/other,Report,18389.0,NaN,"nursing\nAddendum;parents visited, mom attempt...",NaN
4,2002426,24131,117291.0,2174-10-29 00:00:00.000,2174-10-29 14:50:00.000,2174-10-29 15:08:00.000,Nursing/other,Report,20713.0,NaN,1. RESP-Infant is in RA with RR 30-50's. Lungs...,NaN


In [16]:
noteevents2.hadm_id.nunique()

50563

we have 50K examples

## 5. Get target text

In [17]:
noteevents2.loc[noteevents2.category == 'Discharge summary', 'target_text'] = noteevents2.loc[noteevents2.category == 'Discharge summary'].apply(lambda x: get_target_text(x.text, x.headers), axis = 1)

In [18]:
noteevents2[noteevents2.category == 'Discharge summary'].head().target_text

4423                                            NOT FOUND
4424    82 y/o female admitted [**2119-5-4**] for cons...
4425    87 yo F with h/o CHF, COPD on 5 L oxygen at ba...
4426                                            NOT FOUND
4427    Patient presented electively for meningioma re...
Name: target_text, dtype: object

## 6. Drop subjects/hadm_ids that do not have "BRIEF HOSPITAL COURSE" or "HOSPITAL COURSE", or have them both

In [19]:
noteevents2[noteevents2.target_text.isin(['NOT FOUND', 'MULTIPLE FOUND', np.nan])].target_text.value_counts()

target_text
NOT FOUND         14590
MULTIPLE FOUND       34
Name: count, dtype: int64

In [20]:
exclude_hadmids = noteevents2[noteevents2.target_text.isin(['NOT FOUND', 'MULTIPLE FOUND'])].hadm_id.tolist()

In [21]:
noteevents3 = noteevents2[~noteevents2.hadm_id.isin(exclude_hadmids)].reset_index(drop=True)
noteevents3.shape

(1091841, 13)

In [22]:
noteevents3.subject_id.nunique()

28752

In [23]:
noteevents3.hadm_id.nunique()

35939

We have nearly 34K samples!

In [24]:
noteevents3.category.value_counts()

category
Nursing/other        378940
Radiology            251399
Nursing              159197
Physician            104507
ECG                   97046
Discharge summary     35939
Echo                  22721
Respiratory           22476
Nutrition              6785
General                6235
Rehab Services         3722
Social Work            1981
Case Management         742
Consult                  79
Pharmacy                 72
Name: count, dtype: int64

## Generate the Dataset

### Create functions to aggregate data for dataset

In [51]:
def cols2dict(row):
    return {'row_id': row.row_id, 'chartdate': row.chartdate, 'category': row.category, 'description': row.description, 'text': row.text}

def prep_dataset(df_og):
    df = df_og.copy()
    df.loc[df.category.str.lower() != 'discharge summary', 'notes'] = df.loc[df.category.str.lower() != 'discharge summary'].apply(lambda row: cols2dict(row), axis = 1)
    grouped = df.groupby(['subject_id', 'hadm_id']).agg({'notes': lambda x: [i for i in x if not pd.isnull(i)], 'target_text': lambda x: [i for i in x if not pd.isnull(i)][0]}).reset_index()
    # group1 = df.groupby(['subject_id', 'hadm_id']).agg({'notes': lambda x: [i for i in x if not pd.isnull(i)]}).reset_index()
    # print(group1.shape[0])
    # group2 = df[(df.category.str.lower()=='discharge summary') & (~pd.isnull(df.target_text))][['subject_id', 'hadm_id', 'target_text']].reset_index()
    # print(group2.shape[0])
    # grouped = pd.merge(group1, group2, how = 'inner', on = ['hadm_id', 'subject_id'])
    print(grouped.shape)
    return grouped

In [52]:
prepped_df = prep_dataset(noteevents3)

(35939, 4)


In [53]:
print(f"Prepped Data Shape: {prepped_df.shape[0]}\nN unique hadm_ids: {prepped_df.hadm_id.nunique()}")

Prepped Data Shape: 35939
N unique hadm_ids: 35939


In [54]:
prepped_df.head()

,subject_id,hadm_id,notes,target_text
0,9,150750.0,"[{'row_id': 1260715, 'chartdate': '2149-11-09 ...",1. NEUROLOGIC: Left putaminal bleed secondar...
1,10,184167.0,"[{'row_id': 793184, 'chartdate': '2103-07-05 0...",1.) Respiratory: Baby Girl [**Known lastname ...
2,17,161087.0,"[{'row_id': 1260773, 'chartdate': '2135-05-09 ...",Admitted through ER as above and referred to C...
3,19,109235.0,"[{'row_id': 1260781, 'chartdate': '2108-08-06 ...",Patient admitted to the trauma service. Orthop...
4,21,109451.0,"[{'row_id': 1260788, 'chartdate': '2134-09-13 ...",1. Rhythm: pt had 2 episodes of monomorphic VT...


In [57]:
prepped_df.target_text.isna().sum()

0

In [58]:
test = prepped_df[prepped_df.notes.apply(lambda x: len(x))==0]
test

,subject_id,hadm_id,notes,target_text
318,406,174925.0,[],PRIMARY REASON FOR HOSPITALIZATION: Patient is...
434,573,169615.0,[],The patient was admitted to the Newborn Nurser...
624,852,133054.0,[],Pt was admitted through same day admission and...
778,1084,165348.0,[],The patient tolerated his procedure without in...
857,1183,145559.0,[],The patient went to the Operating Room on [**2...
...,...,...,...,...
35614,98562,122148.0,[],84 y/o male with history of COPD presenting wi...
35671,98802,101071.0,[],# Goals of care: The neurosurgery team had a d...
35749,99183,184631.0,[],UPPER GASTROINTESTINAL BLEED: The patient und...
35901,99822,146997.0,[],This is a 61 year old male with Stage IV squam...


#### Seems like there are a couple hundred cases where a patient's only note was a discharge summary for their hospital visit.... let's confirm

##### Confirm via inner join with noteevents table

In [69]:
joined_df = pd.merge(noteevents3, test, how = 'inner', on = ['subject_id', 'hadm_id'])
joined_df.shape[0]

234

Shape is same!

##### Confirm again with iteration... can never be too safe :)

In [67]:
subjs = test.subject_id.tolist()
hadms = test.hadm_id.tolist()
l = list(zip(subjs,hadms))
target_ids = []
for subj, hadm in l:
    tdfs = noteevents3[(noteevents3.subject_id == subj) & (noteevents3.hadm_id==hadm)].shape[0]
    if tdfs>1:
        print('FOUND ANOMALY')
        target_ids.append((subj, hadm, tdfs))
target_ids
    

[]

Confirmed!

### Drop those records

In [70]:
prepped_df = prepped_df[prepped_df.notes.apply(lambda x: len(x))>0].reset_index(drop=True)
prepped_df.shape

(35705, 4)

### Train, validation, test split the data

In [72]:
from sklearn.model_selection import train_test_split

train_ratio = 0.70
validation_ratio = 0.15
test_ratio = 0.15

random_state = 123

train, valtest = train_test_split(prepped_df, test_size=1 - train_ratio, random_state = random_state)
validation, test = train_test_split(valtest, test_size=test_ratio/(test_ratio + validation_ratio), random_state = random_state) 


In [74]:
print(f"Train Shape: {train.shape[0]} ({round(train.shape[0]/prepped_df.shape[0]*100, 3)}% Actual)")
print(f"Validation Shape: {validation.shape[0]} ({round(validation.shape[0]/prepped_df.shape[0]*100, 3)}% Actual)")
print(f"Train Shape: {test.shape[0]} ({round(test.shape[0]/prepped_df.shape[0]*100, 3)}% Actual)")

Train Shape: 24993 (69.999% Actual)
Validation Shape: 5356 (15.001% Actual)
Train Shape: 5356 (15.001% Actual)


### Create a Hugging Face dataset

In [106]:
ds_train = Dataset.from_dict(train, split = 'train')
ds_validation = Dataset.from_dict(validation, split = 'validation')
ds_test = Dataset.from_dict(test, split = 'test')

In [107]:
ds_train

Dataset({
    features: ['subject_id', 'hadm_id', 'notes', 'target_text'],
    num_rows: 24993
})

### Push the datasets to the hub

In [111]:
ds_train.push_to_hub("mimiciii-hospitalcourse")

Pushing dataset shards to the dataset hub: 100%|█████████████████████████████| 3/3 [07:27<00:00, 149.26s/it]


In [112]:
ds_validation.push_to_hub("mimiciii-hospitalcourse")

Pushing dataset shards to the dataset hub: 100%|█████████████████████████████| 1/1 [01:42<00:00, 102.46s/it]


In [113]:
ds_test.push_to_hub("mimiciii-hospitalcourse")

Pushing dataset shards to the dataset hub: 100%|██████████████████████████████| 1/1 [01:36<00:00, 96.65s/it]


### Confirm we can load the datasets

In [115]:
test_load_test_data = load_dataset('dmacres/mimiciii-hospitalcourse', split = 'test')



Generating test split: 100%|██████████████████████████████████| 5356/5356 [00:00<00:00, 14074.66 examples/s]


In [120]:
test_load_test_data

Dataset({
    features: ['subject_id', 'hadm_id', 'notes', 'target_text'],
    num_rows: 5356
})

In [122]:
test_load_val_data = load_dataset('dmacres/mimiciii-hospitalcourse', split = 'validation')
test_load_val_data 

Dataset({
    features: ['subject_id', 'hadm_id', 'notes', 'target_text'],
    num_rows: 5356
})

In [121]:
test_load_train_data = load_dataset('dmacres/mimiciii-hospitalcourse', split = 'train')
test_load_train_data

Dataset({
    features: ['subject_id', 'hadm_id', 'notes', 'target_text'],
    num_rows: 24993
})